In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyomo
import scipy
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.util.infeasible import log_infeasible_constraints
from datetime import datetime,timedelta
import pickle
import json
from sklearn.cluster import KMeans
import logging
import os
pd.set_option('display.max_columns', None)  # or specify a number if you want a limit
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Basic Setting
grid = "369_0"
next_two = True # True consider next 2 trips,False consider only next 1 trip
scenario_year = 2050
weekday_dict = {'Monday':(7,4), 'Tuesday':(7,5),'Wednesday':(7,6),'Thursday':(7,7),'Friday':(7,8),'Saturday':(7,9),'Sunday':(7,10)}
# weekday_dict = {'Monday':(1,3), 'Tuesday':(1,4),'Wednesday':(1,5),'Thursday':(1,6),'Friday':(1,7),'Saturday':(1,8),'Sunday':(1,9)}
# weekday_dict = {'Monday':(1,10), 'Tuesday':(1,11),'Wednesday':(1,12),'Thursday':(1,13),'Friday':(1,14),'Saturday':(1,15),'Sunday':(1,16)}
day_start_ts = pd.to_datetime(f"{scenario_year}-07-08 00:00:00")
day_start = day_start_ts.day
day_end_ts = pd.to_datetime(f"{scenario_year}-07-09 00:00:00")
monitor_hr = int((day_end_ts - day_start_ts).total_seconds()/3600)
weekday = "Friday"
path = f"{grid}/{scenario_year}_{weekday}_07_08_controlled"
sanitycheck_path = f"{path}/sanitycheck"

participate_rate = 1  # 1 no constraints on participation rate, parking event based, not vehicle based
min_power_level = 0.5 # 1 constant power request at max. charger power, otherwise set a lower limit on power requested: min_power_level * max_power
# Calculate the number of groups
num_groups = 30
TP_dict = {1:2,2:2,3:2,4:3,5:3,6:4,7:4,8:4,9:4,10:3,11:2,12:1}
TP = TP_dict[day_start_ts.month] # 1:Dec, 2:Jan,Feb,Mar,Nov, 3:Apr,May,Oct, 4:Jun,Jul,Aug,Sep
os.makedirs(path,exist_ok=True)
os.makedirs(sanitycheck_path,exist_ok=True)

In [3]:
# Data Preprocessing
def calculate_next_trip_e(row):
    if next_two and (row['next_SoE_bc']+(row['next_parking_time']//60*7) < row[f'next_2_travel_TP{TP}_consumption']): # if charge with 7 KW during whole next pakring event could not cover energy requirement for the second next trip charge the part that could not be covered within this parking event
        next_trip_e = row[f'next_travel_TP{TP}_consumption']+ row[f'next_2_travel_TP{TP}_consumption'] - row['next_SoE_bc']+(row['next_parking_time']//60*7)#row['next_2_travel_TP1_consumption']
    else:
        next_trip_e =  row[f'next_travel_TP{TP}_consumption']
    return next_trip_e

def get_hour_index(ts):
    idx = int(divmod((ts-day_start_ts).total_seconds(),3600)[0]) 
    return idx

def get_soe_init(SoE_bc,arr_idx):
    if arr_idx<=0:
        arr_idx = 1
    soe_init_list = [0]*monitor_hr
    for idx in range(arr_idx):
        soe_init_list[idx] = SoE_bc
    return soe_init_list


def create_dict(start_ts,end_ts,start_hour_idx,end_hour_idx):
    # if end_hour_idx>monitor_hr-1:
    #     end_hour_idx = monitor_hr-1
    dict = [0]*monitor_hr
    for hour in range(monitor_hr):
        if start_hour_idx<hour and end_hour_idx>hour:
            dict[hour] = 60
        elif start_hour_idx==hour and end_hour_idx==hour:
            dict[hour] = end_ts.minute-start_ts.minute
        elif start_hour_idx<hour and end_hour_idx==hour:
            dict[hour] = end_ts.minute
        elif start_hour_idx==hour and end_hour_idx>hour:
            dict[hour] = 60-start_ts.minute
    return dict

def create_charge_time_list(charge_decision, start_ts, end_ts, start_hour_idx,end_hour_idx):
    # if end_hour_idx>monitor_hr-1:
    #     end_hour_idx = monitor_hr-1
    dict = [0]*monitor_hr
    if charge_decision:
        for hour in range(monitor_hr):
            if start_hour_idx<hour and end_hour_idx>hour:
                dict[hour] = 60
            elif start_hour_idx==hour and end_hour_idx==hour:
                dict[hour] = end_ts.minute-start_ts.minute
            elif start_hour_idx<hour and end_hour_idx==hour:
                dict[hour] = end_ts.minute
            elif start_hour_idx==hour and end_hour_idx>hour:
                dict[hour] = 60-start_ts.minute
    return dict


# Optimization
def create_sparse_dict(value_list,monitior_hr):
    value_indices = value_list.index.to_numpy()
    values = np.array(value_list.tolist())
    row_indices = np.repeat(value_indices,monitor_hr)
    col_indices = np.tile(np.arange(monitior_hr),len(value_indices))
    data_values = values.flatten()

    # Filter out zero values to keep the matrix sparse
    non_zero_mask = data_values != 0
    row_indices = row_indices[non_zero_mask].astype(int)
    col_indices = col_indices[non_zero_mask].astype(int)
    data_values = data_values[non_zero_mask]
    
    sparse_matrix = scipy.sparse.coo_matrix((data_values,(row_indices,col_indices)), shape=(max(value_indices)+1,monitor_hr))
    return sparse_matrix

    
# Data Postprocessing
def get_timestamp_pair(row):
    process = {}
    process_key = ()
    power = []
    for hour in range(monitor_hr):
        p_t = row['optimized_power_list'][hour]
        min_t = row['hourly_time_dict'][hour]
        if hour>0:
            min_pre, p_pre = row['hourly_time_dict'][hour-1],row['optimized_power_list'][hour-1]
        else: 
            min_pre,p_pre = 0,0
        if hour<(monitor_hr-1):
            min_next, p_next = row['hourly_time_dict'][hour+1],row['optimized_power_list'][hour+1]
        else:
            min_next, p_next = 0,0
        if p_pre==0 and p_t!=0:
            if (min_t<=60) and (min_pre>0):
                start_min = 0
            elif (min_pre==0):
                start_min = 60-min_t
            # elif (min_t == 60 or min_pre==60) and (hour!=row['arr_time_idx']):
            #     start_min = 0
            # elif (min_t<60 and min_t>0):# and (hour!=row['arr_time_idx']): #and min_pre!=60 
            #     start_min = 60-min_t
            # elif (hour==row['arr_time_idx']):
            #     start_min = row['arr_time'].minute

            else:
                start_min = 0

            start_delta_hr = day_start_ts + timedelta(hours=hour)
            start_ts = pd.Timestamp(datetime(year=start_delta_hr.year,month=start_delta_hr.month,day=start_delta_hr.day,hour=start_delta_hr.hour,minute=start_min))
            process_key = (start_ts,)
        if p_t!=0:
            power.append(p_t)
        if p_next==0 and p_t!=0:
            if (min_t==60):
                end_min=59
            elif p_pre!=0 and min_t<60:
                end_min = min_t
            elif p_pre==0 and min_t<60:
                end_min = start_min+min_t-1
            # if (min_t==60 or min_next>0) and (hour!=row['park_end_time_idx']):
            #     end_min=59
            # elif (min_t<60 and min_next==0 and min_pre>0) and (hour!=row['park_end_time_idx']):
            #     end_min=min_t
            # elif (hour==row['park_end_time_idx']):
            #     end_min=row['park_end_time'].minute
            # else:
            #     end_min = 0

            end_delta_hr = day_start_ts + timedelta(hours=hour)
            end_ts = pd.Timestamp(datetime(year=end_delta_hr.year,month=end_delta_hr.month,day=end_delta_hr.day,hour=end_delta_hr.hour,minute=end_min))

            process_key = process_key + (end_ts,)
            process[process_key] = power
            process_key = ()
            power = []
    return process

def check_outside(row):
    if (row['process_cnt']==0) and (row['st_chg_time']==row['ed_chg_time']):
        return False
    elif row['process_cnt']>=1 and (((next(iter(row['process_list'].keys())))[0]>=row['st_chg_time']) and ((next(iter(row['process_list'].keys())))[1]<=row['ed_chg_time'])):
        return False
    elif row['process_cnt']>=1 and (((next(iter(row['process_list'].keys())))[0]>=row['ed_chg_time']) or ((next(iter(row['process_list'].keys())))[1]<=row['st_chg_time'])):
        return True
    else:
        return True

def correct_st_chg_time(row):
    if row['process_list'] and row['optimized_process_mean_power']>0:
        if row['c']==True and row['st_chg_time']<day_start_ts and row['ed_chg_time']>day_start_ts and next(iter(row['process_list'].keys()))[0]==day_start_ts:
            return row['st_chg_time']
        elif next(iter(row['process_list'].keys()))[0]>=day_start_ts:
            return next(iter(row['process_list'].keys()))[0]
    elif row['process_list'] and row['optimized_process_mean_power']<0:
        return next(iter(row['process_list'].keys()))[0]
    elif (row['c']==True and row['st_chg_time']<day_start_ts and row['ed_chg_time']<day_start_ts) or (row['c']==True and row['st_chg_time']>=day_end_ts and row['ed_chg_time']>=day_end_ts):
        return row['st_chg_time']
    else:
        return None
    
def correct_ed_chg_time(row):
    if row['process_list'] and row['optimized_process_mean_power']>0:
        if row['c']==True and row['ed_chg_time']>=day_end_ts-timedelta(minutes=1) and row['st_chg_time']<day_end_ts-timedelta(minutes=1) and next(iter(row['process_list'].keys()))[1]>=day_end_ts-timedelta(minutes=1) and row['day_end_soe']<row['B']:
            return row['ed_chg_time']
        elif next(iter(row['process_list'].keys()))[1]<=day_end_ts-timedelta(minutes=1):
            return next(iter(row['process_list'].keys()))[1]
    elif row['process_list'] and row['optimized_process_mean_power']<0:
        return next(iter(row['process_list'].keys()))[1]
    elif (row['c']==True and row['st_chg_time']<day_start_ts and row['ed_chg_time']<day_start_ts) or (row['c']==True and row['st_chg_time']>=day_end_ts and row['ed_chg_time']>=day_end_ts):
        return row['ed_chg_time']
    else:
        return None

In [ ]:
df = pd.read_csv("/Users/huiwen/Library/Mobile Documents/com~apple~CloudDocs/Thesis/mobility/grid369_mobility_dataset.csv")
for col in ['dep_time','arr_time','st_chg_time','ed_chg_time']:
    df[col] = pd.to_datetime(df[col],format='mixed')
df['chg_time'] = pd.to_timedelta(df['ed_chg_time']-df['st_chg_time'],unit='m')

df['dep_time'] = df.apply(lambda row:row['dep_time'].replace(year=scenario_year,month=weekday_dict[row['type_day']][0],day=weekday_dict[row['type_day']][1]), axis=1)
df['dep_time_idx'] = df.apply(lambda x:get_hour_index(x['dep_time']),axis=1)

df['arr_time'] = df['dep_time'] + pd.to_timedelta(df['trav_time'], unit='m')
df['arr_hour'] = df['arr_time'].dt.hour
df['arr_time_idx'] = df.apply(lambda x:get_hour_index(x['arr_time']), axis=1)
df['arr_day'] = df['arr_time'].dt.day

df['park_end_time'] = df['arr_time']+pd.to_timedelta(df['parking_time'],unit='m')
df['park_end_hour'] = df['park_end_time'].dt.hour
df['park_end_time_idx'] = df.apply(lambda x:get_hour_index(x['park_end_time']), axis=1)
df['park_end_day'] = df['park_end_time'].dt.day

df['st_chg_time']=df['arr_time']
df['st_chg_time_idx'] = df.apply(lambda x:get_hour_index(x['st_chg_time']),axis=1)
df['ed_chg_time'] = df['st_chg_time']+df['chg_time']
df['ed_chg_time_idx'] = df.apply(lambda x:get_hour_index(x['ed_chg_time']),axis=1)
df['chg_time'] = df['chg_time'].dt.total_seconds()/60

df.sort_values(by=['person','dep_time'])
df_byperson = df.groupby('person')
df[f'next_travel_TP{TP}_consumption'] = df_byperson[f'TP{TP} consumption kWh'].shift(-1).fillna(0)
df['next_parking_time'] = df_byperson['parking_time'].shift(-1).fillna(0)
df[f'next_2_travel_TP{TP}_consumption'] = df_byperson[f'TP{TP} consumption kWh'].shift(-2).fillna(0)
df['next_SoE_bc'] = df_byperson['SoE_bc'].shift(-1).fillna(0)
df['next_trip_e'] = df.apply(calculate_next_trip_e,axis=1)



df.drop(['next_parking_time', 'st_mun_name', 'st_canton_name', 'st_urb_type',
         'st_mnt_type', 'ed_mun_name', 'ed_canton_name', 'ed_urb_type',
         'ed_mnt_type', 'Yearly kWh'], axis=1, inplace=True) 
# 'next_travel_TP1_consumption','next_2_travel_TP1_consumption', 'TP1 rate kWh/100 km','TP2 rate kWh/100 km', 'TP2 consumption kWh','TP3 rate kWh/100 km', 'TP3 consumption kWh', 'TP4 rate kWh/100 km','TP4 consumption kWh',


d = df[(df['park_end_time']>=day_start_ts) & (df['arr_time']<day_end_ts)].copy()
d['hourly_time_dict'] = d.apply(lambda x:create_dict(x.arr_time,x.park_end_time,x.arr_time_idx,x.park_end_time_idx), axis =1)
d['charge_time_list'] = d.apply(lambda x:create_charge_time_list(x.c,x.st_chg_time,x.ed_chg_time,x.st_chg_time_idx,x.ed_chg_time_idx), axis=1)
d['charge_power_list'] = d.apply(lambda x: [x['chg rate'] if t >30 else 0 for t in x['charge_time_list']], axis=1)
d['charge_power_list_sanity'] = d.apply(lambda x: [x['chg rate'] if t >0 else 0 for t in x['charge_time_list']], axis=1)
d['charge_energy_list'] = d.apply(lambda x:[t/60*x['chg rate']for t in x['charge_time_list']], axis=1)
d['charge_energy_sum'] = d['charge_energy_list'].apply(sum)
# d['is_first'] = ~d.duplicated(subset='person', keep='first')
# d['is_first'] = d['is_first'].astype(bool)
d['augmented_SoE_bc'] = np.where(
    (d['st_chg_time'] < day_start_ts) & (d['ed_chg_time'] > day_start_ts),
    d['SoE_ac'] - d['charge_energy_sum'],
    np.where(
        (d['st_chg_time'] < day_start_ts) & (d['ed_chg_time'] < day_start_ts),
        d['SoE_ac'],
        d['SoE_bc']
    )
)
d['soe_init'] = d.apply(lambda x:get_soe_init(x.augmented_SoE_bc,x.arr_time_idx), axis=1)
d['real_chg_e'] = d['SoE_ac']-d['augmented_SoE_bc']
d.insert(0,'event_index',d.index)

d.to_pickle(f'{path}/grid_{grid}_{scenario_year}_{day_start_ts}_{monitor_hr}_preprocessed_2trip.pkl')


In [ ]:
with open(f'{path}/grid_{grid}_{scenario_year}_{day_start_ts}_{monitor_hr}_preprocessed_2trip.pkl','rb') as d:
    d = pickle.load(d)

In [ ]:
# Step 1: Calculate aggregates for arr_hour and dep_hour for each person_id
times = d.groupby('person').agg({
    'arr_time_idx':['min','max'],
    'dep_time_idx':['min','max']
}).reset_index()
# Flatten the MultiIndex for columns created by aggregation
times.columns = ['_'.join(col).strip('_') for col in times.columns.values]
# Create a feature dataframe for clustering
feature_df = times[['arr_time_idx_min', 'arr_time_idx_min', 'dep_time_idx_min', 'dep_time_idx_max']]

# Step 2: Cluster based on behavior
kmeans = KMeans(n_clusters=30, random_state=42).fit(feature_df)
times['cluster'] = kmeans.labels_
# Assign cluster labels to each person
clustered = pd.merge(d, times[['person', 'cluster']], on='person')
clustered = clustered.sort_values(by=['person','dep_time_idx','cluster'])
# Get unique persons and the start index for each person
unique_persons = clustered['person'].unique()
person_start_index = {person: i for i, person in enumerate(unique_persons)}


# Create a dictionary to hold the group number for each person
person_to_group = {}
# Assign groups in a round-robin fashion
group_number = 1
for person in unique_persons:
    person_to_group[person] = group_number
    group_number = group_number % num_groups + 1  # Loop back to 1 after reaching num_groups
# Now assign the group to each row in the DataFrame based on the 'person' column
clustered['group'] = clustered['person'].map(person_to_group)
clustered.drop(['cluster'],axis=1,inplace=True)
clustered['pre_SoE_ac'] = clustered.groupby(['group','person', 'grid'],dropna=False)['SoE_ac'].shift(1).fillna(clustered['augmented_SoE_bc'])
clustered['SoE_change'] = clustered['pre_SoE_ac']-clustered['augmented_SoE_bc']
clustered['next_dep_time_idx'] = clustered.groupby('person')['dep_time_idx'].shift(-1).fillna(monitor_hr-1)
clustered['next_SoE_change'] = clustered.groupby('person')['SoE_change'].shift(-1).fillna(0)

clustered.drop(['pre_SoE_ac'],axis=1)
clustered.set_index('event_index', inplace=True)
clustered.to_pickle(f'{path}/grid_{grid}_{scenario_year}_{day_start_ts}_{monitor_hr}_2trip_clustered_{num_groups}.pkl')


In [4]:
# Normalize Tobia's Nexus Output
hv_bus = str(89)
# Controlled charging
charge = pd.read_csv(f"/Users/huiwen/Library/Mobile Documents/com~apple~CloudDocs/Thesis/extracted_data/map_bus/Added_up_charge_{scenario_year}_raw.csv") # in MW
charge['ts'] = pd.to_datetime(charge['ts'])
charge = charge.loc[(charge.ts<day_end_ts) & (charge.ts>=day_start_ts)][['ts','peak',hv_bus]]
# Controlled discharging
discharge = pd.read_csv(f"/Users/huiwen/Library/Mobile Documents/com~apple~CloudDocs/Thesis/extracted_data/map_bus/EVBatt_power_hourly_{scenario_year}_discharge_mapped.csv")
discharge = discharge.rename(columns={'Unnamed: 0':'ts'})
discharge['ts'] = pd.to_datetime(discharge['ts'])
discharge = discharge.loc[(discharge.ts<day_end_ts) & (discharge.ts>=day_start_ts)][['ts',hv_bus]]
# Find Netload Max
net= charge['89']-discharge['89']
period_max = net.max()
charge['normalized_profile'] = charge[hv_bus]/period_max
charge.index=range(monitor_hr)
discharge['normalized_profile'] = discharge[hv_bus]/period_max
discharge.index=range(monitor_hr)
net = charge['89']-discharge['89']
net_normalized = net/period_max


In [5]:
def shape_matching(i,path):
    m = pyo.ConcreteModel()

    ## Functions
    def initialize_max_p_rule(m,park,person,arr_idx,end_idx):
        return MAX_P_dict[park]
    def initialize_soe_change_rule(m,park,person,arr_idx,end_idx):
        return SOE_CHANGE_dict[park]
    def p_bound_rule(m,park,person,arr_idx,end_idx,t):
        if (park,person,arr_idx,end_idx,t) not in PARKHR_dict.keys() or (PARKHR_dict[(park,person,arr_idx,end_idx,t)]<=0):
            return (0,0)
        else:
            return (0,MAX_P_dict[park])
    def initialize_next_e_rule(m,park,person,arr_idx,end_idx):
        return NEXT_E_dict[park]
    def soe_bound_rule(m,person,t):
        return (0,EVCAP_dict[person])
    def soe_init_rule(m,person,t):
        if t==0:
            return SOE_init_dict[person]
        else:
            return 0

    ## Const
    M = 1e8
    PARK_CNT = len(cluster)
    unshifted_daily_net_charge = normalized_tot_e

    ## Set
    m.EVBAT = pyo.Set(initialize=cluster.person.unique())
    m.PARK = pyo.Set(dimen=4,initialize = park_tuple) # (Parking Event,EVBAT,Bus) Set
    m.T = pyo.Set(initialize=list(range(monitor_hr)))

    ## Params
    m.CHARGE_TARGET = pyo.Param(m.T,initialize=charge_to_match_dict)
    m.DISCHARGE_TARGET = pyo.Param(m.T,initialize=discharge_to_match_dict)
    m.SOE_init = pyo.Param(m.EVBAT,initialize=SOE_init_dict)
    m.MAX_P = pyo.Param(m.PARK,initialize=initialize_max_p_rule)
    m.SOE_CHANGE = pyo.Param(m.PARK,initialize=initialize_soe_change_rule)
    m.PARKHR = pyo.Param(m.PARK,m.T,initialize=PARKHR_dict,default=0)
    m.PPLANED = pyo.Param(m.PARK,m.T,initialize=P_PLANED_dict,default=0)
    m.EVCAP = pyo.Param(m.EVBAT,initialize=EVCAP_dict)
    m.NEXT_E = pyo.Param(m.PARK,initialize=initialize_next_e_rule)

    ## Vars
    m.SOE = pyo.Var(m.EVBAT,m.T,within=pyo.NonNegativeReals,bounds=soe_bound_rule,initialize=soe_init_rule)#
    m.CHARGE_P = pyo.Var(m.PARK, m.T,within=pyo.NonNegativeReals,bounds=p_bound_rule,initialize=0) 
    m.DISCHARGE_P = pyo.Var(m.PARK,m.T,within=pyo.NonNegativeReals,bounds=p_bound_rule,initialize=0)
    m.IS_CHARGING = pyo.Var(m.PARK,m.T,within=pyo.Binary,initialize=0)
    m.IS_DISCHARGING = pyo.Var(m.PARK,m.T,within=pyo.Binary,initialize=0)
    m.IS_ACTIVE = pyo.Var(m.PARK,within=pyo.Binary,initialize=0)
    m.CHARGE_JUMP = pyo.Var(m.PARK,m.T, within=pyo.Binary,initialize=0) # detect charge jump
    m.DISCHARGE_JUMP = pyo.Var(m.PARK,m.T, within=pyo.Binary,initialize=0) # detect discharge jump
    m.FLEX = pyo.Var(m.PARK,m.T, within=pyo.Binary,initialize=0) # detect flexibility for each person

    ## Slack/Auxiliary
    m.POWER_SHIFT_ABS = pyo.Var(m.PARK,m.T,within=pyo.NonNegativeReals)
    m.BUFFER = pyo.Var(m.T,within=pyo.NonNegativeReals,initialize=0)
    m.ENERGY_DEFICIT = pyo.Var(within=pyo.NonNegativeReals,bounds=(0,0.3*unshifted_daily_net_charge),initialize=0) #Room for daily net charge energy to deviate
    m.ENERGY_SURPLUS = pyo.Var(within=pyo.NonNegativeReals,bounds=(0,0.3*unshifted_daily_net_charge),initialize=0) #Room for daily net charge energy to deviate
    m.TOT_CHARGE = pyo.Var(m.T,within=pyo.NonNegativeReals,initialize=0) #Auxiliary Variable for hourly tot charge power
    m.TOT_DISCHARGE = pyo.Var(m.T,within=pyo.NonNegativeReals,initialize=0) #Auxiliary Variable for hourly tot discharge power


    ############################
    def identify_charging_rule(m,park,person,arr_idx,end_idx,t):
        """
        enforce is_charging=1 if charging, 0 if discharging/no action
        """
        return m.CHARGE_P[park,person,arr_idx,end_idx,t] <= M * m.IS_CHARGING[park,person,arr_idx,end_idx,t]
    m.IdentifyCharging = pyo.Constraint(m.PARK, m.T, rule=identify_charging_rule)


    def identify_discharging_rule(m,park,person,arr_idx,end_idx,t):
        """
        enforce is_discharging=1 if discharging, 0 if charging/no action
        """
        return m.DISCHARGE_P[park,person,arr_idx,end_idx,t] <= M * m.IS_DISCHARGING[park,person,arr_idx,end_idx,t]
    m.IdentifyDischarging = pyo.Constraint(m.PARK, m.T, rule=identify_discharging_rule)


    def non_simultaneous_rule(m,park,person,arr_idx,end_idx,t):
        """
        Avoid simultaneous charge and discharge
        """
        return m.IS_CHARGING[park,person,arr_idx,end_idx,t] + m.IS_DISCHARGING[park,person,arr_idx,end_idx,t] <= 1
    m.NonSimultaneous = pyo.Constraint(m.PARK, m.T, rule=non_simultaneous_rule)


    def decreasing_charge_rule(m,park,person,arr_idx,end_idx,t):
        """
        Charging power monotonically decreasing
        """
        if t>=arr_idx and t<(monitor_hr-1):
            return (m.CHARGE_P[park,person,arr_idx,end_idx,t]-m.CHARGE_P[park,person,arr_idx,end_idx,t+1])*m.IS_CHARGING[park,person,arr_idx,end_idx,t]>=0
        else:
            return pyo.Constraint.Skip
    m.DecreaseCharging = pyo.Constraint(m.PARK,m.T,rule=decreasing_charge_rule)


    def decreasing_discharge_rule(m,park,person,arr_idx,end_idx,t):
        """
        Discharging power monotonically decreasing
        """
        if t>=arr_idx and t<(monitor_hr-1):
            return (m.DISCHARGE_P[park,person,arr_idx,end_idx,t]-m.DISCHARGE_P[park,person,arr_idx,end_idx,t+1])*m.IS_DISCHARGING[park,person,arr_idx,end_idx,t]>=0
        else:
            return pyo.Constraint.Skip
    m.DecreaseDischarging = pyo.Constraint(m.PARK,m.T,rule=decreasing_discharge_rule)

    def soe_update_rule(m,person,t):
        """
        Update SOE 
        """
        if t==0:
            return m.SOE[person,0]==m.SOE_init[person]
        else:
            return m.SOE[person,t]==(m.SOE[person,t-1]
                                    +sum(m.CHARGE_P[park,person,arr_idx,end_idx,t-1]*m.PARKHR[park,person,arr_idx,end_idx,t-1] for park,p,arr_idx,end_idx in m.PARK if p==person)
                                    -sum(m.DISCHARGE_P[park,person,arr_idx,end_idx,t-1]*m.PARKHR[park,person,arr_idx,end_idx,t-1] for park,p,arr_idx,end_idx in m.PARK if p==person)
                                    -sum(m.SOE_CHANGE[park,p,arr_idx,end_idx] for park,p,arr_idx,end_idx in m.PARK if (p==person and arr_idx==t))
                                    )
    m.SoeUpdate = pyo.Constraint(m.EVBAT,m.T,rule=soe_update_rule)

    def soe_end_rule(m,person):
        return (m.SOE[person,monitor_hr-1]
                +sum(m.CHARGE_P[park,person,arr_idx,end_idx,monitor_hr-1]*m.PARKHR[park,person,arr_idx,end_idx,monitor_hr-1] for park,p,arr_idx,end_idx in m.PARK if p==person)
                -sum(m.DISCHARGE_P[park,person,arr_idx,end_idx,monitor_hr-1]*m.PARKHR[park,person,arr_idx,end_idx,monitor_hr-1] for park,p,arr_idx,end_idx in m.PARK if p==person)
                )<=m.EVCAP[person]
    m.SoeEnd = pyo.Constraint(m.EVBAT,rule=soe_end_rule)

    # def soe_upper_rule(m,person,t):
    #     """
    #     SOE upper soft limit
    #     """
    #     return m.SOE[person,t]<=m.EVCAP[person]
    # m.SoeUpper = pyo.Constraint(m.EVBAT,m.T,rule=soe_upper_rule)

    def next_trip_rule(m,park,person,arr_idx,end_idx,t):
        """
        SOE requirement for next trip
        """
        if end_idx==t-1:
            return m.SOE[person,t]>=m.NEXT_E[park,person,arr_idx,end_idx]
        else:
            return pyo.Constraint.Skip  
    m.NextTrip = pyo.Constraint(m.PARK,m.T,rule=next_trip_rule)

    def detect_charge_jump(m,park,person,arr_idx,end_idx,t):
        """
        Detect start of charging process
        """
        if t==0:
            return m.CHARGE_JUMP[park,person,arr_idx,end_idx,t]==1*m.IS_CHARGING[park,person,arr_idx,end_idx,t]
        else:
            return m.IS_CHARGING[park,person,arr_idx,end_idx,t]-m.IS_CHARGING[park,person,arr_idx,end_idx,t-1] <= M * m.CHARGE_JUMP[park,person,arr_idx,end_idx,t]
    m.DetectChargeJump = pyo.Constraint(m.PARK,m.T,rule=detect_charge_jump)

    def charge_jump_rule(m,park,p,arr_idx,end_idx):
        """
        max. 1 start of charging is allowed
        """
        return sum(m.CHARGE_JUMP[park,p,arr_idx,end_idx,t] for t in m.T )<=1
    m.ChargeJump = pyo.Constraint(m.PARK,rule=charge_jump_rule)

    def detect_discharge_jump(m,park,person,arr_idx,end_idx,t):
        """
        Detect start of charging process
        """
        if t==0:
            return m.DISCHARGE_JUMP[park,person,arr_idx,end_idx,t]==1*m.IS_DISCHARGING[park,person,arr_idx,end_idx,t]
        else:
            return m.IS_DISCHARGING[park,person,arr_idx,end_idx,t]-m.IS_DISCHARGING[park,person,arr_idx,end_idx,t-1] <= M * m.DISCHARGE_JUMP[park,person,arr_idx,end_idx,t]
    m.DetectDischargeJump = pyo.Constraint(m.PARK,m.T,rule=detect_discharge_jump)

    def discharge_jump_rule(m,park,p,arr_idx,end_idx):
        """
        max. 1 start of discharging is allowed
        """
        return sum(m.DISCHARGE_JUMP[park,p,arr_idx,end_idx,t] for t in m.T )<=1
    m.DishargeJump = pyo.Constraint(m.PARK,rule=discharge_jump_rule)

    def only_charge_or_discharge_rule(m,park,p,arr_idx,end_idx):
        """
        Only charge or discharge is allowed for each parking event
        """
        return sum(m.CHARGE_JUMP[park,p,arr_idx,end_idx,t] for t in m.T )+sum(m.DISCHARGE_JUMP[park,p,arr_idx,end_idx,t] for t in m.T )<= 1
    m.OneJump = pyo.Constraint(m.PARK,rule=only_charge_or_discharge_rule)


    def min_charge_time_rule(m,park,p,arr_idx,end_idx):
        """
        Be active for at least 1 hr
        """
        return sum(m.IS_CHARGING[park,p,arr_idx,end_idx,t] * m.PARKHR[park,p,arr_idx,end_idx,t] + m.IS_DISCHARGING[park,p,arr_idx,end_idx,t] * m.PARKHR[park,p,arr_idx,end_idx,t] for t in m.T) >= 1 * m.IS_ACTIVE[park,p,arr_idx,end_idx]
    m.MinActiveDuration = pyo.Constraint(m.PARK, rule=min_charge_time_rule)

    def activity_rule(m,park,p,arr_idx,end_idx):
        """
        Identify wether there's action decided in a parking event
        """
        return sum(m.IS_CHARGING[park,p,arr_idx,end_idx,t]*m.PARKHR[park,p,arr_idx,end_idx,t]+m.IS_DISCHARGING[park,p,arr_idx,end_idx,t]*m.PARKHR[park,p,arr_idx,end_idx,t] for t in m.T)<=M*m.IS_ACTIVE[park,p,arr_idx,end_idx]
    m.DetectAcitivity = pyo.Constraint(m.PARK,rule=activity_rule)

    def power_shift_rule_1(m,park,p,arr_idx,end_idx,t):
        """
        Create the slack m.POWER_SHIFT_ABS to identify wether the charging is shfited
        """
        return m.POWER_SHIFT_ABS[park,p,arr_idx,end_idx,t]>=(m.CHARGE_P[park,p,arr_idx,end_idx,t]-m.DISCHARGE_P[park,p,arr_idx,end_idx,t])-m.PPLANED[park,p,arr_idx,end_idx,t]
    m.PowerShiftAbs1 = pyo.Constraint(m.PARK,m.T,rule=power_shift_rule_1)
        
    def power_shift_rule_2(m,park,p,arr_idx,end_idx,t):
        """
        Create the slack m.POWER_SHIFT_ABS to identify wether the charging is shfited
        """
        return m.POWER_SHIFT_ABS[park,p,arr_idx,end_idx,t]>=-((m.CHARGE_P[park,p,arr_idx,end_idx,t]-m.DISCHARGE_P[park,p,arr_idx,end_idx,t])-m.PPLANED[park,p,arr_idx,end_idx,t])
    m.PowerShiftAbs2 = pyo.Constraint(m.PARK,m.T,rule=power_shift_rule_2)

    def plug_in_cnt_rule(m,t):
        """
        Count plug in number at each m.T
        """
        return sum(1 if m.PARKHR[park,person,arr_idx,end_idx,t]>0 else 0 for park,person,arr_idx,end_idx in m.PARK)
    m.PlugInCnt = pyo.Expression(m.T,rule=plug_in_cnt_rule)

    def decide_flex_rule(m,park,person,arr_idx,end_idx,t):
        """
        Determine value for m.FLEX
        """
        return m.POWER_SHIFT_ABS[park,person,arr_idx,end_idx,t] <= M*m.FLEX[park,person,arr_idx,end_idx,t]
    m.DecideFlex = pyo.Constraint(m.PARK,m.T,rule=decide_flex_rule)

    def participate_rule(m,t):
        """
        Flexibility particiaption limit
        """
        return sum(m.FLEX[park,person,arr_idx,end_idx,t] for park,person,arr_idx,end_idx in m.PARK)<=0.29*m.PlugInCnt[t] + m.BUFFER[t]
    m.ParticipateLimit = pyo.Constraint(m.T, rule=participate_rule)


    def net_charge_daily(m):
        """
        Sum of daily net charging energy
        """
        return sum((m.CHARGE_P[park,person,arr_idx,end_idx,t]-m.DISCHARGE_P[park,person,arr_idx,end_idx,t])*m.PARKHR[park,person,arr_idx,end_idx,t] for park,person,arr_idx,end_idx in m.PARK for t in m.T)
    m.NetChargeDaily = pyo.Expression(rule=net_charge_daily)

    def match_daily_energy_rule(m):
        """
        Match daily net charge energy as close as possible
        """
        return m.NetChargeDaily + m.ENERGY_DEFICIT == unshifted_daily_net_charge + m.ENERGY_SURPLUS
    m.MatchDailyNetCharge = pyo.Constraint(rule=match_daily_energy_rule)

    def hourly_charge_power_sum_rule(m,t):
        return sum(m.CHARGE_P[park,person,arr_idx,end_idx,t] * m.IS_CHARGING[park,person,arr_idx,end_idx,t] for park,person,arr_idx,end_idx in m.PARK)
    m.HourlyChargedPowerSum = pyo.Expression(m.T,rule=hourly_charge_power_sum_rule)
    def mask_tot_charge_z(m,t):
        return m.TOT_CHARGE[t]==m.HourlyChargedPowerSum[t]
    m.TotCharge = pyo.Constraint(m.T, rule=mask_tot_charge_z)

    def hourly_discharge_power_sum_rule(m,t):
        return sum(m.DISCHARGE_P[park,person,arr_idx,end_idx,t] * m.IS_DISCHARGING[park,person,arr_idx,end_idx,t] for park,person,arr_idx,end_idx in m.PARK)
    m.HourlyDischargedPowerSum = pyo.Expression(m.T,rule=hourly_discharge_power_sum_rule)
    def mask_tot_discharge_z(m,t):
        return m.TOT_DISCHARGE[t]==m.HourlyDischargedPowerSum[t]
    m.TotDischarge = pyo.Constraint(m.T, rule=mask_tot_discharge_z)

    ###########################
    # Objective 
    def objective_rule(m):
        return sum((m.CHARGE_TARGET[t]-m.TOT_CHARGE[t])**2 for t in m.T) + sum((m.DISCHARGE_TARGET[t]-m.TOT_DISCHARGE[t])**2 for t in m.T) + 10000*sum(m.BUFFER[t] for t in m.T)+0.1*sum(m.POWER_SHIFT_ABS[park,person,arr_idx,end_idx,t] for park,person,arr_idx,end_idx in m.PARK for t in m.T)
    m.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


    ###########################
    # Solve model
    solver = pyo.SolverFactory('gurobi')
    # solver.options['FeasibilityTol'] = 1e-2
    # solver.options['OptimalityTol'] = 1e-2
    solver.options['MIPGapAbs'] = 0.7
    solver.options['NoRelHeurTime'] = 30
    solver.options['NodeMethod'] = 2
    solver.options['MIPFocus'] = 1
    # solver.options['TimeLimit'] = 300
    solver.solve(m,tee=True,logfile=f"{path}/cluster_{i}.log")   #,keepfiles=True,logfile="match_profile_log.log")
    logging.getLogger().setLevel(logging.INFO)
    log_infeasible_constraints(m)

    ###############################
    # Save Results
    ch_dict = {(park, t): m.CHARGE_P[park,person,arr_idx,end_idx, t].value*emob_max_p for park,person,arr_idx,end_idx in m.PARK for t in m.T} # Denormalize back to normal power value in kW
    dis_dict = {(park,t):m.DISCHARGE_P[park,person,arr_idx,end_idx,t].value*emob_max_p for park,person,arr_idx,end_idx in m.PARK for t in m.T}
    soe_dict = {(person,t):pyo.value(m.SOE[person,t]*emob_max_p) for person in m.EVBAT for t in m.T}
    flex_dict = {(person,t): m.FLEX[park,person,arr_idx,end_idx,t].value for park,person,arr_idx,end_idx in m.PARK for t in m.T}
    ch = pd.Series(ch_dict).unstack()
    ch.to_csv(f'{path}/{grid}_cluster_{i}_charge.csv')
    dis = pd.Series(dis_dict).unstack()
    dis.to_csv(f'{path}/{grid}_cluster_{i}_discharge.csv')
    soe = pd.Series(soe_dict).unstack()
    soe.to_csv(f'{path}/{grid}_cluster_{i}_soe.csv')
    person_flag = pd.Series(flex_dict).unstack()
    person_flag.to_csv(f"{path}/{grid}_cluster_{i}_person_participate_flag.csv")

    return 0

In [ ]:
# Read from local result
with open(f"{path}/grid_{grid}_{scenario_year}_{day_start_ts}_{monitor_hr}_2trip_clustered_{num_groups}.pkl","rb") as c:
    clustered = pickle.load(c)
clustered = clustered[(clustered['grid']==grid)]

for i in range(num_groups):

    cluster = clustered[(clustered['group']==i+1) & (clustered['grid']==grid)]
    emob_agg_e = [sum(x) for x in zip(*cluster['charge_energy_list'])] # energy in kWh
    emob_agg_p = [sum(x) for x in zip(*cluster['charge_power_list'])] # power in kW
    emob_max_p, emob_min_p,emob_tot_p = max(emob_agg_p), min(emob_agg_p), sum(emob_agg_p)
    emob_max_e, emob_min_e, emob_tot_e= max(emob_agg_e), min(emob_agg_e),sum(emob_agg_e)
    normalized_tot_e = emob_tot_e/emob_max_p
    print(f"Start Cluster {i}")
    print(emob_max_p)
    if emob_max_p!=0:
        """
        Prepare Model Input Data
        """
        emob_agg_p_norm =  [p/emob_max_p for p in emob_agg_p]
        cluster.loc[:,'normalized_chg_power'] = cluster['chg rate']/emob_max_p
        
        charge_to_match_dict = charge.normalized_profile.to_dict()
        discharge_to_match_dict = discharge.normalized_profile.to_dict()

        SOE_init_dict = (cluster.groupby('person')['augmented_SoE_bc'].first()/emob_max_p).to_dict()
        PARK_to_PERSON_dict = cluster.person.to_dict()
        PARK_to_ENDIDX_dict = cluster.park_end_time_idx.to_dict() # key:parking event -> value: park_end_time_idx
        PARK_to_ARRIDX_dict = cluster.arr_time_idx.to_dict() # key:parking event -> value: arr_time_idx
        park_tuple = [(PARK,int(PARK_to_PERSON_dict[PARK]),max(PARK_to_ARRIDX_dict[PARK],0),min(int(PARK_to_ENDIDX_dict[PARK]),monitor_hr-1)) for PARK,PERSON in  PARK_to_PERSON_dict.items()]
        MAX_P_dict = (cluster['chg rate']/emob_max_p).to_dict()
        SOE_CHANGE_dict = (cluster.SoE_change/emob_max_p).to_dict()
        NEXT_E_dict = (cluster.next_trip_e/emob_max_p).to_dict()
        PARKHR_dict = {(PARK,int(PARK_to_PERSON_dict[PARK]),max(PARK_to_ARRIDX_dict[PARK],0),min(int(PARK_to_ENDIDX_dict[PARK]),monitor_hr-1),t):cluster.loc[PARK].hourly_time_dict[t]/60 for PARK in cluster.index for t in range(monitor_hr)}
        EVCAP_dict = (cluster.groupby('person').B.first()/emob_max_p).to_dict()
        
        P_PLANED_coo = create_sparse_dict(cluster.charge_power_list_sanity,monitor_hr)
        P_PLANED_dict = {(park,int(PARK_to_PERSON_dict[park]),max(PARK_to_ARRIDX_dict[park],0),min(int(PARK_to_ENDIDX_dict[park]),monitor_hr-1),t):power/emob_max_p for (park,t,power) in zip(P_PLANED_coo.row,P_PLANED_coo.col,P_PLANED_coo.data)}
        PARKHR_coo = create_sparse_dict(cluster.hourly_time_dict,monitor_hr)
        PARKHR_dict = {(park,int(PARK_to_PERSON_dict[park]),max(PARK_to_ARRIDX_dict[park],0),min(int(PARK_to_ENDIDX_dict[park]),monitor_hr-1),t):minutes/60 for  park,t,minutes in zip(PARKHR_coo.row,PARKHR_coo.col,PARKHR_coo.data)}

        opt_res_code = shape_matching(i,path)
    else:
        index = list(cluster.index)
        T = list(range(monitor_hr))
        res_dict = {(e, t):0 for e in index for t in T} # Denormalize back to normal power value in kW
        # Convert the dictionary into a multi-index series to facilitate unstacking
        res = pd.Series(res_dict).unstack()
        # Save the restructured data to CSV
        res.to_csv(f'{path}/369_0_cluster_{i}.csv')
        
        

In [ ]:
agg_e = [sum(x) for x in zip(*clustered['charge_energy_list'])] # energy in kWh
agg_p = [sum(x) for x in zip(*clustered['charge_power_list'])] # power in kW

max_p,min_p,tot_p = max(agg_p), min(agg_p),sum(agg_p)
max_e,min_e,tot_e= max(agg_e), min(agg_e),sum(agg_e)

In [ ]:
concat_charge = pd.DataFrame()
concat_discharge = pd.DataFrame()
concat_soe = pd.DataFrame()
concat_flexperson = pd.DataFrame()
for i in range(num_groups):
    charge_i = pd.read_csv(f'{path}/{grid}_cluster_{i}_charge.csv')
    concat_charge = pd.concat([concat_charge,charge_i],axis=0)
    discharge_i = pd.read_csv(f'{path}/{grid}_cluster_{i}_discharge.csv')
    concat_discharge = pd.concat([concat_discharge,discharge_i],axis=0)
    # soe_i = pd.read_csv(f'{path}/{grid}_cluster_{i}_soe.csv')
    # concat_soe = pd.concat([concat_soe,soe_i],axis=0)
    # flexperson_i = pd.read_csv(f'{path}/{grid}_cluster_{i}_person_participate_flag.csv')
    # concat_flexperson = pd.concat([concat_flexperson,flexperson_i],axis=0)
    
concat_charge = concat_charge.rename(columns={'Unnamed: 0':'event_index'})
concat_discharge = concat_discharge.rename(columns={'Unnamed: 0':'event_index'})
concat_soe = concat_soe.rename(columns={'Unnamed: 0':'person'})
# concat_flexperson = concat_flexperson.rename(columns={'Unnamed: 0':'person'})

concat_res = concat_charge-concat_discharge
concat_res.loc[:,'event_index'] = concat_charge['event_index']
concat_charge.set_index('event_index',inplace=True)
concat_discharge.set_index('event_index',inplace=True)
concat_res.set_index('event_index', inplace=True)


# Cast to 0 is power too low
for column in concat_res.columns:
    if column != 'event_index':
        # Apply the condition and replace values
        concat_res.loc[:, column] = concat_res[column].apply(lambda x: 0 if abs(x) < 0.01 else x)
        
concat_charge.to_csv(f'{path}/concat_charge_power_all.csv')
concat_discharge.to_csv(f'{path}/concat_dicharge_power_all.csv')
concat_res.to_csv(f'{path}/concat_net_power_all.csv')
concat_soe.to_csv(f'{path}/concat_soe_all.csv')
concat_flexperson.to_csv(f'{path}/concat_flexperson_all.csv')

concat_sum = concat_res.sum().sum()
concat_max = concat_res.sum().max()
plt.figure(figsize=(10, 6))

# Optimized results with solid lines and markers
plt.plot(concat_res.sum()/concat_max, label='Optimized Net Charge Normalized', color='blue', linestyle='-', marker='o')
plt.plot(concat_charge.sum()/concat_max, label='Optimized Charge Normalized', color='green', linestyle='-', marker='^')
plt.plot(concat_discharge.sum()/concat_max, label='Optimized Discharge Normalized', color='red', linestyle='-', marker='s')

# Nexus outputs with dashed lines
plt.plot(net_normalized, label='Nexus Net Charge Normalized', color='blue', linestyle='--')
plt.plot(charge['normalized_profile'], label='Nexus Charge Normalized', color='green', linestyle='--')
plt.plot(discharge['normalized_profile'], label='Nexus Discharge Normalized', color='red', linestyle='--')

plt.title('Comparison of Nexus Outputs and Optimization Results')
plt.xlabel('Time')
plt.ylabel('Normalized Value')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig(f'{path}/optimized_norm_power_vs_nexus.png')
plt.show()


In [ ]:
res_p = concat_res.sum()
res_c = concat_charge.sum()
res_d = concat_discharge.sum()
plt.plot(res_p,label='optimized_net_power_profile')
plt.plot(agg_p, label='mobility_data_power_profile')
plt.plot(res_c, label='optimized_charge_profile')
plt.plot(res_d,label='optimized_discharge_profile')
plt.legend()
plt.savefig(f'{path}/optimized_vs_mobility_power_profile.png')

In [ ]:
concat_res = concat_res.sort_index()
concat_charge = concat_charge.sort_index()
concat_discharge = concat_discharge.sort_index()
time = pd.DataFrame(clustered['hourly_time_dict'].sort_index())
event_e = concat_res.apply(lambda row: [row[col]*time.at[row.name,'hourly_time_dict'][col]/60 for col in range(monitor_hr)],axis=1)
charge_e = concat_charge.apply(lambda row: [row[col]*time.at[row.name,'hourly_time_dict'][col]/60 for col in range(monitor_hr)],axis=1)
discharge_e = concat_discharge.apply(lambda row: [row[col]*time.at[row.name,'hourly_time_dict'][col]/60 for col in range(monitor_hr)],axis=1)

net_e = pd.DataFrame(event_e.tolist())
net_e.set_index(concat_res.index,inplace=True)
res_net_e = pd.Series(net_e.sum(),name='agg_net_e')
res_net_e_sum = res_net_e.sum()

charge_e = pd.DataFrame(charge_e.tolist())
charge_e.set_index(concat_charge.index,inplace=True)
res_charge_e = pd.Series(charge_e.sum(),name='agg_charge_e')
res_charge_e_sum = res_charge_e.sum()

discharge_e = pd.DataFrame(discharge_e.tolist())
discharge_e.set_index(concat_discharge.index,inplace=True)
res_discharge_e = pd.Series(discharge_e.sum(),name='agg_charge_e')
res_discharge_e_sum = res_discharge_e.sum()


plt.plot(res_net_e,label='optimized_net_energy_profile')
# plt.plot(res_charge_e, label='optimized_charge_energy_profile')
# plt.plot(res_discharge_e, label='optimized_discharge_energy_profile')
plt.plot(agg_e,label='mobility_data_energy_profile')

plt.text(5,6000,f'mobility:{tot_e:.2f}')
plt.text(5,5500,f'optimized_net:{res_net_e_sum:.2f}')
plt.text(5,5000,f'optimized_charge:{res_charge_e_sum:.2f}')
plt.text(5,4500,f'optimized_discharge:{res_discharge_e_sum:.2f}')
plt.legend()
plt.savefig(f'{path}/optimized_vs_mobility_net_reqeusted_energy_profile.png')

In [ ]:
res = pd.read_csv(f'{path}/concat_net_power_all.csv',index_col='event_index')
powerlist = res.apply(lambda row: [round(x,1) for x in row.tolist()], axis=1)
powerlist.sort_index()
# soe_all = pd.read_csv(f"{path}/concat_soe_all.csv", index_col='person')
# soe_end = soe_all.iloc[:,-1]

In [ ]:
def update_SoE_bc(group):
    if len(group)>1:
        # Initialize the first row of 'new_col' with the value from 'column1'
        group['update_SoE_bc'].iloc[0] = (group['day_start_SoE'].iloc[0]-group['SoE_change'].iloc[0])
        # For subsequent rows, add 'column1' from the previous row and 'column2' from the previous row
        for i in range(1, len(group)):
            group['update_SoE_bc'].iloc[i] = group['update_SoE_bc'].iloc[i-1] + (group['optimized_charge_energy_sum'].iloc[i-1]-group['SoE_change'].iloc[i])
    else:
        group['update_SoE_bc'].iloc[0]=group['day_start_SoE'].iloc[0]-group['SoE_change'].iloc[0]
    return group


d_in = clustered.copy()
d_in.sort_index()
d_in.loc[:,'optimized_power_list'] = powerlist
d_in.loc[:,'optimized_energy_list'] = d_in.apply(lambda row: [row['hourly_time_dict'][x]/60 * row['optimized_power_list'][x] for x in range(monitor_hr)], axis=1)
d_in.loc[:,'optimized_charge_energy_sum'] = d_in['optimized_energy_list'].apply(sum)

d_in['process_list'] = d_in.apply(lambda row:get_timestamp_pair(row),axis=1)
d_in['optimized_process_mean_power'] = d_in['process_list'].apply(lambda x: np.mean(next(iter(x.values())))if len(x)>0 else 0)
d_in['augmented_SoE_ac'] = d_in.apply(lambda row:row['augmented_SoE_bc']+row['charge_energy_sum'] if row['ed_chg_time']>=day_end_ts else row['SoE_ac'],axis=1)
d_in['augmented_SoE_ac'] =  np.where(
    (d_in['st_chg_time'] < day_end_ts) & (d_in['ed_chg_time'] >=day_end_ts),
    d_in['augmented_SoE_bc'] + d_in['charge_energy_sum'],
    np.where(
        (d_in['st_chg_time'] >= day_end_ts) & (d_in['ed_chg_time'] >= day_end_ts),
        d_in['augmented_SoE_bc'],
        d_in['SoE_ac']
    )
)
d_in['unshifted_day_end_soe'] = d_in.groupby('person')['augmented_SoE_ac'].transform(lambda x: x.iloc[-1])
d_in['process_cnt'] = d_in['process_list'].apply(len)
d_in['shifted_process_cnt'] = d_in.groupby('person')['process_cnt'].transform(lambda x:x.sum()) # per person
d_in['unshifted_process_cnt'] = d_in.groupby('person')['c'].transform(lambda x: x.sum()) # per person
d_in['optimized_power_list'] = d_in['optimized_power_list'].apply(lambda x: [round(float(i),1) for i in x])
d_in['charge_power_list_sanity'] = d_in['charge_power_list_sanity'].apply(lambda x: [round(float(i),1) for i in x])
d_in['flex'] = d_in['optimized_power_list'] != d_in['charge_power_list_sanity'] # per event
d_in['outside_base'] = d_in.apply(lambda row:check_outside(row), axis=1) # per event
d_in = d_in.sort_values(by=['person', 'dep_time'])
d_in.loc[:,'day_start_SoE'] = d_in['person'].map(d_in.groupby('person')['augmented_SoE_bc'].first())
d_in['update_SoE_bc'] = pd.NA
d_in = d_in.groupby('person', group_keys=False).apply(update_SoE_bc)
d_in['update_SoE_ac'] = d_in['update_SoE_bc'] + d_in['optimized_charge_energy_sum']
d_in['day_end_soe'] = d_in.groupby('person')['update_SoE_ac'].transform(lambda x:x.iloc[-1])
d_in['day_end_soe_shift'] = d_in['day_end_soe']-d_in['unshifted_day_end_soe']
d_in['shifted_st_chg_time'] = d_in.apply(lambda row:correct_st_chg_time(row),axis=1)
d_in['shifted_ed_chg_time'] = d_in.apply(lambda row:correct_ed_chg_time(row),axis=1)
d_in.to_pickle(f'{path}/grid_{grid}_matched_{day_start_ts}_{monitor_hr}_{scenario_year}.pkl')

In [ ]:
(d_in.groupby('person')['unshifted_day_end_soe'].first()-d_in.groupby('person')['day_end_soe'].first()).sum()

In [ ]:
d_in['SoE_more'] = d_in['update_SoE_ac']-d_in['B']
len(d_in.loc[d_in['SoE_more']>1])

In [ ]:
with open(f"{path}/grid_{grid}_matched_{day_start_ts}_{monitor_hr}_{scenario_year}.pkl", "rb") as file:
    d_in = pickle.load(file)

In [ ]:
# Stats
# Participation rate
flex_d_in = d_in[d_in['flex']==True].copy()
print("Person/Vehicle count:", d_in['person'].nunique())
print("Parking event count:", len(d_in))
print("Person/Vehicel participation count:",flex_d_in['person'].nunique())
print("Parking event participation count:",len(flex_d_in))
print("Flexibility participation rate by person:", float(flex_d_in['person'].nunique()/d_in['person'].nunique()))
print("Flexibility participation rate by parking event", float(len(flex_d_in)/len(d_in)))


# Shifted Day End soe within EVs participated in the flexibility
flex_d_in['day_end_soe_change'] = flex_d_in['day_end_soe']-flex_d_in['unshifted_day_end_soe']
soe_change =  flex_d_in.groupby('person')['day_end_soe_change'].first()
max_change,min_change,sum_change_net = soe_change.max(), soe_change.min(), soe_change.sum()
print("Net Total Day end SoE change:",sum_change_net)
max_bin = (max(abs(max_change),abs(min_change))//10+1)*10
soe_change_bins = np.arange(-max_bin, max_bin + 10, 10)

# Adjusting the plots to have the boxplot lying down (horizontal) for alignment with the barplot's values
fig, axs = plt.subplots(2, 1, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 2]}, sharex=True)
q1, q2, q3 = np.percentile(soe_change, [25, 50, 75])

# Horizontal boxplot for 'soe_shift' on the top subplot
axs[0].boxplot(soe_change, vert=False, patch_artist=True, positions=[1])
axs[0].set_title('Day end SoE shift')
axs[0].set_yticks([1])
axs[0].set_yticklabels(['Value'])
# Adding quartile labels to the boxplot
axs[0].text(q1-30, 1.2, f"Q1: {q1:.2f}", ha='center', va='center', fontsize=10, color='blue', backgroundcolor='white')
axs[0].text(q2, 1.2, f"Median: {q2:.2f}", ha='center', va='center', fontsize=10, color='red', backgroundcolor='white')
axs[0].text(q3+30, 1.2, f"Q3: {q3:.2f}", ha='center', va='center', fontsize=10, color='blue', backgroundcolor='white')

# Barplot for frequency data on the bottom subplot, aligned with the boxplot's values
counts,edges,bars = axs[1].hist(soe_change, color='skyblue', edgecolor='black',bins=soe_change_bins)
# counts,edges,bars = axs[1].hist(soe_shift, color='skyblue', edgecolor='black',bins=20,density=True)
axs[1].set_title('Frequency Count for Day End SOE Shift')
axs[1].set_xticks(soe_change_bins)

axs[1].set_ylabel('Count')
plt.bar_label(bars)
plt.tight_layout()
plt.savefig(f"{path}/sanitycheck/day_end_soeshift.png")

In [ ]:
# # Save Flexibility Participation Flag for Following Days
# if following_days:
#     flex_dict_new = d_in.groupby('person')['flex'].any().astype(int).to_dict()
#     # len(list(set(flex_dict.keys())-set(flex_dict_new.keys())))
#     flex_dict_update = flex_dict.copy()
#     flex_dict_update.update(flex_dict_new)
#     with open(f'{path}/flex_dict.json', 'w') as file:
#         json.dump(flex_dict_update, file)
# else:
#     flex_dict = d_in.groupby('person')['flex'].any().astype(int).to_dict()
#     with open(f'{path}/flex_dict.json', 'w') as file:
#         json.dump(flex_dict,file)

# count_of_ones = sum(1 for value in flex_dict_update.values() if value == 1)
# print(count_of_ones,len(flex_dict_update))

# with open(f'{path}/flex_dict.json', 'w') as file:
#     json.dump(flex_dict_update, file)

# # Find common keys using set intersection
# common_keys = set(flex_dict_new.keys()) & set(flex_dict_update.keys())

# # Create a dictionary to see the values from both dictionaries for these keys
# common_items = {key: (flex_dict_new[key], flex_dict_update[key]) for key in common_keys}


# different_items = {}
# for key in flex_dict_update:
#     if key in flex_dict_update and flex_dict_update[key] != flex_dict_new[key]:
#         different_items[key] = (flex_dict_new[key], flex_dict_update[key])
# print(different_items)

In [ ]:
all = d_in.copy()
# Shifted Day End soe within EVs participated in the flexibility
all['day_end_soe_change'] = all['day_end_soe']-all['unshifted_day_end_soe']
soe_change_all =  all.groupby('person')['day_end_soe_change'].first()
max_change_all,min_change_all,sum_change_net_all = soe_change_all.max(), soe_change_all.min(), soe_change_all.sum()
print("Net Total Day end SoE change:",sum_change_net)
max_bin_all = (max(abs(max_change_all),abs(min_change_all))//10+1)*10
soe_change_bins_all = np.arange(-max_bin_all, max_bin_all + 10, 10)

# Adjusting the plots to have the boxplot lying down (horizontal) for alignment with the barplot's values
fig, axs = plt.subplots(2, 1, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 2]}, sharex=True)
q1, q2, q3 = np.percentile(soe_change, [25, 50, 75])

# Horizontal boxplot for 'soe_shift' on the top subplot
axs[0].boxplot(soe_change_all, vert=False, patch_artist=True, positions=[1])
axs[0].set_title('Day end SoE shift')
axs[0].set_yticks([1])
axs[0].set_yticklabels(['Value'])
# Adding quartile labels to the boxplot
axs[0].text(q1-30, 1.2, f"Q1: {q1:.2f}", ha='center', va='center', fontsize=10, color='blue', backgroundcolor='white')
axs[0].text(q2, 1.2, f"Median: {q2:.2f}", ha='center', va='center', fontsize=10, color='red', backgroundcolor='white')
axs[0].text(q3+30, 1.2, f"Q3: {q3:.2f}", ha='center', va='center', fontsize=10, color='blue', backgroundcolor='white')

# Barplot for frequency data on the bottom subplot, aligned with the boxplot's values
counts,edges,bars = axs[1].hist(soe_change_all, color='skyblue', edgecolor='black',bins=soe_change_bins_all)
# counts,edges,bars = axs[1].hist(soe_shift, color='skyblue', edgecolor='black',bins=20,density=True)
axs[1].set_title('Frequency Count for Day End SOE Shift all Vehicles')
axs[1].set_xticks(soe_change_bins_all)

axs[1].set_ylabel('Count')
plt.bar_label(bars)
plt.tight_layout()
plt.savefig(f"{path}/sanitycheck/day_end_soeshift_all_vehicles.png")

In [ ]:
# Day End SoC change
day_end_soc = d_in.groupby('person')[['day_end_soe','unshifted_day_end_soe','B']].first()

def calc_2d_hist_percentage(x_data, y_data, bins):
    heatmap, xedges, yedges = np.histogram2d(x_data, y_data, bins=[bins, bins])
    total_counts = np.sum(heatmap)
    percentage_heatmap = (heatmap / total_counts) * 100  # Convert counts to percentage
    return percentage_heatmap, xedges, yedges

fig, axs = plt.subplots(1, 1, figsize=(6,6))
bins = np.arange(0,110,10)
SoC_change_percentage, SoC_xedges, SoC_yedges = calc_2d_hist_percentage(day_end_soc['day_end_soe']/day_end_soc['B']*100, day_end_soc['unshifted_day_end_soe']/day_end_soc['B']*100, bins)
cax = axs.pcolormesh(SoC_xedges, SoC_yedges, SoC_change_percentage.T, shading='auto', cmap='viridis')  # Transpose to correct the axis orientation
axs.set_title('Shift Effect on Day End SoC of single EV')
axs.set_xlabel('Day end SoC after shifting')
axs.set_ylabel('Day end SoC without shifting')
axs.set_aspect('equal')  # Set aspect ratio to be equal
fig.colorbar(cax, ax=axs, label='Percentage (%)')
plt.savefig(f"{path}/sanitycheck/single_ev_day_end_soc_shift.png")


In [ ]:
# Flex charge start SoC
charge_data = d_in[d_in['optimized_process_mean_power'] > 0].copy()
discharge_data = d_in[d_in['optimized_process_mean_power'] < 0].copy()
unshifted_charge_data = d_in[d_in['charge_energy_sum'] > 0].copy()

bins = np.arange(0, 110, 10)

# Create subplots
fig, axs = plt.subplots(3, 1, figsize=(5,9))

# Function to calculate 2D histogram data and convert to percentage
def calc_2d_hist_percentage(x_data, y_data, bins):
    heatmap, xedges, yedges = np.histogram2d(x_data, y_data, bins=[bins, bins])
    total_counts = np.sum(heatmap)
    percentage_heatmap = (heatmap / total_counts) * 100  # Convert counts to percentage
    return percentage_heatmap, xedges, yedges

# Calculate 2D histogram percentages
charge_percentage, xedges, yedges = calc_2d_hist_percentage(charge_data['update_SoE_ac']/charge_data['B']*100, charge_data['update_SoE_bc']/charge_data['B']*100, bins)
discharge_percentage, _, _ = calc_2d_hist_percentage(discharge_data['update_SoE_ac']/discharge_data['B']*100, discharge_data['update_SoE_bc']/discharge_data['B']*100, bins)
unshifted_charge_percentage, _, _ = calc_2d_hist_percentage(unshifted_charge_data['SoE_ac']/unshifted_charge_data['B']*100, unshifted_charge_data['SoE_bc']/unshifted_charge_data['B']*100, bins)
# Plot heatmaps for unshifted charging
cax = axs[0].pcolormesh(xedges, yedges, unshifted_charge_percentage.T, shading='auto', cmap='viridis')  # Transpose to correct the axis orientation
axs[0].set_title('SoC for unshifted Charging')
axs[0].set_xlabel('State of Charge After (%)')
axs[0].set_ylabel('State of Charge Before (%)')
axs[0].set_aspect('equal')  # Set aspect ratio to be equal
fig.colorbar(cax, ax=axs[0], label='Percentage (%)')

# Plot heatmaps for shifted charging
cax = axs[1].pcolormesh(xedges, yedges, charge_percentage.T, shading='auto', cmap='viridis')
axs[1].set_title('SoC for Shifted Charging')
axs[1].set_xlabel('State of Charge After (%)')
axs[1].set_ylabel('State of Charge Before (%)')
axs[1].set_aspect('equal')
fig.colorbar(cax, ax=axs[1], label='Percentage (%)')


# Plot heatmaps for shifted discharging
cax = axs[2].pcolormesh(xedges, yedges, discharge_percentage.T, shading='auto', cmap='viridis')
axs[2].set_title('SoC for Shifted Discharging')
axs[2].set_xlabel('State of Charge After (%)')
axs[2].set_ylabel('State of Charge Before (%)')
axs[2].set_aspect('equal')
fig.colorbar(cax, ax=axs[2], label='Percentage (%)')

# Layout adjustments
plt.tight_layout()
plt.savefig(f'{path}/sanitycheck/SoC_before_after.png')


In [ ]:
out_base = d_in[d_in['outside_base']==True].copy()
out_base['ch_flag'] = out_base['optimized_process_mean_power'].apply(lambda x: 1 if x>0 else 0)
out_base['dis_flag'] = out_base['optimized_process_mean_power'].apply(lambda x: 1 if x<0 else 0)
res = out_base.groupby('person').agg(ch_flag_sum = ('ch_flag','sum'),dis_flag_sum=('dis_flag','sum'))

def label_bars(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate('{}'.format(height),
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 1.5),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
# Calculating frequency counts for each column
ch = res['ch_flag_sum'].value_counts().sort_index()
dis = res['dis_flag_sum'].value_counts().sort_index()

# Ensuring both series have the same index to avoid plotting issues
all_indices = ch.index.union(dis.index)
ch = ch.reindex(all_indices, fill_value=0)
dis = dis.reindex(all_indices, fill_value=0)

# Plotting frequency counts side by side
ind = np.arange(len(all_indices))  # the x locations for the groups
width = 0.3  # the width of the bars

fig, ax = plt.subplots()
bars_ch = ax.bar(ind - width/2, ch, width, label='charge_cnt', color='SkyBlue')
bars_dis = ax.bar(ind + width/2, dis, width, label='discharge_cnt', color='IndianRed')

label_bars(bars_ch)
label_bars(bars_dis)
# Adding some text for labels, title, and custom x-axis tick labels, etc.
ax.set_ylabel('Counts')
ax.set_title('Distribution of Charge and Discharge times outside base profile per person')
ax.set_xticks(ind)
ax.set_xticklabels(all_indices)
ax.legend()
plt.savefig(f"{path}/sanitycheck/ch_dis_times_outside_baseprofile.png")

In [ ]:
# Start charge hour 
ch = d_in[d_in['optimized_process_mean_power']>0].copy()
ch['st_chg_hour'] = ch['shifted_st_chg_time'].apply(lambda x:x.strftime("%Y-%m-%d %H"))
ch['ed_chg_hour'] = ch['shifted_ed_chg_time'].apply(lambda x:x.strftime("%Y-%m-%d %H"))
ch['chg_durat'] = ch.apply(lambda row: (next(iter(row['process_list']))[1]-next(iter(row['process_list']))[0]).total_seconds()/3600, axis=1)

dis = d_in[d_in['optimized_process_mean_power']<0].copy()
dis['st_chg_hour'] = dis['shifted_st_chg_time'].apply(lambda x:x.strftime("%Y-%m-%d %H"))
dis['ed_chg_hour'] = dis['shifted_ed_chg_time'].apply(lambda x:x.strftime("%Y-%m-%d %H"))
dis['chg_durat'] = dis.apply(lambda row: (next(iter(row['process_list']))[1]-next(iter(row['process_list']))[0]).total_seconds()/3600, axis=1)

charge_st_stat = ch.groupby('st_chg_hour').size().to_frame('cnt')
charge_st_stat['prob'] = charge_st_stat['cnt']/charge_st_stat['cnt'].sum()
charge_ed_stat = ch.groupby('ed_chg_hour').size().to_frame('cnt')
charge_ed_stat['prob'] = charge_ed_stat['cnt']/charge_ed_stat['cnt'].sum()

dis_st_stat = dis.groupby('st_chg_hour').size().to_frame('cnt')
dis_st_stat['prob'] = dis_st_stat['cnt']/dis_st_stat['cnt'].sum()
dis_ed_stat = dis.groupby('ed_chg_hour').size().to_frame('cnt')
dis_ed_stat['prob'] = dis_ed_stat['cnt']/dis_ed_stat['cnt'].sum()

fig, axs = plt.subplots(3, 1, figsize=(10,20))
axs[0].hist(ch['chg_durat'])
axs[0].set_title('Charge Duration Probability Distribution')
axs[1].bar(charge_st_stat.index.astype(str),charge_st_stat['prob'])
axs[1].tick_params(axis='x',labelrotation=90)
axs[1].set_title('Start Charging Time Probability Distribution')
axs[2].bar(charge_ed_stat.index.astype(str),charge_ed_stat['prob'])
axs[2].tick_params(axis='x',labelrotation=90)
axs[2].set_title('End Charging Time Probability Distribution')
plt.tight_layout()
plt.savefig(f"{path}/sanitycheck/charge_time_stats.png")

fig, axs = plt.subplots(3, 1, figsize=(10,20))
axs[0].hist(dis['chg_durat'])
axs[0].set_title('Discharge Duration Probability Distribution')
axs[1].bar(dis_st_stat.index.astype(str),dis_st_stat['prob'])
axs[1].tick_params(axis='x',labelrotation=90)
axs[1].set_title('Start Discharging Time Probability Distribution')
axs[2].bar(dis_ed_stat.index.astype(str),dis_ed_stat['prob'])
axs[2].tick_params(axis='x',labelrotation=90)
axs[2].set_title('End Discharging Time Probability Distribution')
plt.tight_layout()
plt.savefig(f"{path}/sanitycheck/discharge_time_stats.png")

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 6))
flex_occur_place = flex_d_in.groupby('end_activity_type').size().to_frame('place_cnt')
flex_occur_place_sorted = flex_occur_place.sort_values('place_cnt',ascending=True)
axs.barh(flex_occur_place_sorted.index,flex_occur_place_sorted['place_cnt'])
for index, value in enumerate(flex_occur_place_sorted['place_cnt']):
    plt.text(value+20, index,str(value))
plt.title('Flexibility occuring place type')
plt.savefig(f"{path}/sanitycheck/flex_occuring_place_type.png")

In [ ]:
flex_ch_hour = (
    d_in[d_in['optimized_process_mean_power']>0].groupby(
        d_in['shifted_st_chg_time'].dt.strftime('%Y-%m-%d %H')
    ).size().to_frame('hour_cnt')
)
flex_dis_hour = (
    d_in[d_in['optimized_process_mean_power']<0].groupby(
        d_in['shifted_st_chg_time'].dt.strftime('%Y-%m-%d %H')
    ).size().to_frame('hour_cnt')
)
unshifted_hour = (
    d_in[d_in['c']==True].groupby(
        d_in['st_chg_time'].dt.strftime('%Y-%m-%d %H')
    ).size().to_frame('hour_cnt')
)

# Merge the data frames on their indices (time), which are formed by the groupby operation
combined = pd.merge(flex_ch_hour, flex_dis_hour, left_index=True, right_index=True, how='outer', suffixes=('_charge', '_discharge'))
combined = pd.merge(combined, unshifted_hour, left_index=True, right_index=True, how='outer', suffixes=('', '_unshifted'))
combined.fillna(0, inplace=True)  # Fill NaN with 0 for missing data
combined.sort_index(inplace=True)  # Sort by index after merging

# Plotting
fig, axs = plt.subplots(figsize=(12, 6))

# Set the width of the bars
bar_width = 0.25

# Get indices for bar positions
indices = range(len(combined))

# Plotting the bars
axs.bar(indices, combined['hour_cnt_charge'], width=bar_width, alpha=0.7, color='blue', label='Shifted Charge Start Hour')
axs.bar([i + bar_width for i in indices], combined['hour_cnt_discharge'], width=bar_width, alpha=0.7, color='red', label='Shifted Discharge Start Hour')
axs.bar([i + 2 * bar_width for i in indices], combined['hour_cnt'], width=bar_width, alpha=0.7, color='green', label='Unshifted Start Hour')

# Adding labels and title
axs.set_title('Flexibility Occurrence by Hour: Charge vs. Discharge vs. Unshifted')
axs.set_xlabel('Time (YYYY-MM-DD HH)')
axs.set_ylabel('Count of Occurrences')

# Set custom x-ticks to center them under the group of bars
plt.xticks([i + bar_width for i in indices], combined.index, rotation=90)

# Adding a legend
axs.legend()

# Adding grid lines
axs.grid(True)


plt.title('Compare Start Hour')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig(f"{path}/sanitycheck/start_hour.png")
